<a href="https://colab.research.google.com/github/wonjun0804/Crime_Predictor/blob/Data-preprocessing/%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터 전처리 과정

In [14]:
#서울시 휴게음식점 인허가 정보.csv 파일에서 "영업상태명"이 "폐업"이나 "취소/말소/만료/정지/중지"인 데이터는 제거하고, 사업장명에 "cu", "씨유", "gs", "지에스", "세븐일레븐", "이마트24", "미니스톱"이 포함된 편의점만 남긴다음 서울 자치구 25개로 나눠서 업소 수를 합산

import pandas as pd

# 1. CSV 파일 불러오기
df = pd.read_csv("서울시 휴게음식점 인허가 정보.csv", encoding="utf-8", low_memory=False)
df.head()


# 2. 제거할 영업상태명 정의
drop_statuses = ['폐업', '취소/말소/만료/정지/중지']

# 3. 비영업 데이터 제거
df = df[~df['영업상태명'].isin(drop_statuses)].copy()

# 4. 편의점 브랜드 키워드 정의 (모두 소문자로 처리)
keywords = ['cu', '씨유', 'gs', '지에스', '세븐일레븐', '이마트24', '미니스톱']

# 5. 사업장명을 소문자로 변환
df['사업장명_소문자'] = df['사업장명'].astype(str).str.lower()

# 6. 키워드 중 하나라도 포함된 경우만 남기기
df = df[df['사업장명_소문자'].apply(lambda x: any(k in x for k in keywords))].copy()

# 7. 자치구 추출 함수 정의
def extract_gu(address):
    try:
        if pd.notnull(address) and "구" in address:
            return address.split()[1]  # 예: 서울특별시 강남구 → 강남구
        else:
            return None
    except:
        return None

# 8. 도로명주소에서 자치구 추출
df['자치구'] = df['도로명주소'].apply(extract_gu)

# 9. 도로명주소에서 못 뽑은 경우 지번주소에서 추출
df['자치구'] = df['자치구'].fillna(df['지번주소'].apply(extract_gu))

# 10. 자치구별 편의점 수 집계
gu_counts = df['자치구'].value_counts().reset_index()
gu_counts.columns = ['자치구', '편의점수']

# 11. 결과 확인 (선택)
print(gu_counts)

# 12. CSV로 저장 (선택)
gu_counts.to_csv("서울시_자치구별_편의점시설.csv", index=False, encoding="utf-8-sig")

     자치구  편의점수
0    강남구   602
1    송파구   432
2    강서구   361
3    서초구   358
4   영등포구   345
5    마포구   340
6    관악구   310
7    강동구   298
8   동대문구   288
9    광진구   275
10   은평구   274
11   구로구   262
12   노원구   245
13    중구   237
14   중랑구   234
15   성북구   228
16   금천구   227
17   동작구   226
18  서대문구   214
19   양천구   207
20   강북구   206
21   성동구   205
22   도봉구   183
23   종로구   181
24   용산구   165


In [15]:
#서울시 숙박업 인허가 정보.csv 파일에서 "영업상태명"이 '폐업'인 데이터만 삭제

# 1. CSV 파일 불러오기 (한글 인코딩 고려: cp949 사용)
df = pd.read_csv("서울시 숙박업 인허가 정보.csv", encoding="cp949")

# 2. 폐업된 숙박업소 제거
# '영업상태명' 컬럼에서 값이 '폐업'이 아닌 행만 필터링
df_filtered = df[df['영업상태명'] != '폐업'].copy()

# 3. 자치구 추출 함수 정의
# 도로명주소 또는 지번주소에서 "강남구", "서초구" 등의 자치구를 추출
def extract_gu(address):
    try:
        if pd.notnull(address) and "구" in address:
            return address.split()[1]  # '서울특별시 강남구 도곡로' → '강남구'
        else:
            return None
    except:
        return None

# 4. 도로명주소에서 자치구 추출
df_filtered['자치구'] = df_filtered['도로명주소'].apply(extract_gu)

# 5. 도로명주소에서 못 뽑은 경우 → 지번주소에서 자치구 추출
df_filtered['자치구'] = df_filtered['자치구'].fillna(df_filtered['지번주소'].apply(extract_gu))

# 6. 자치구별 숙박업소 개수 세기
# 자치구 기준으로 value_counts() 또는 groupby().size() 사용
gu_counts = df_filtered['자치구'].value_counts().reset_index()
gu_counts.columns = ['자치구', '숙박업소수']

# 7. 결과 확인 (선택)
print(gu_counts)

# 8. 결과 저장 (선택)
gu_counts.to_csv("서울시_자치구별_숙박업수.csv", index=False, encoding="utf-8-sig")

     자치구  숙박업소수
0     중구    376
1    종로구    255
2   영등포구    222
3    강남구    131
4   동대문구    131
5    강북구    124
6    구로구    122
7    관악구    104
8    강서구    104
9    중랑구     89
10  서대문구     89
11   용산구     82
12   송파구     76
13   마포구     72
14   강동구     69
15   금천구     68
16   광진구     66
17   성동구     62
18   은평구     62
19   성북구     59
20   서초구     38
21   동작구     37
22   노원구     28
23   도봉구     22
24   양천구     13


In [16]:
#서울시 청소년게임제공업 인허가 정보.csv 파일에서 "영업상태명"이 "폐업"이나 "취소/말소/만료/정지/중지"인 데이터는 제거하고, 서울 자치구 25개로 나눠서 업소 수를 합산
import pandas as pd

# 1. CSV 파일 불러오기
df = pd.read_csv("서울시 청소년게임제공업 인허가 정보.csv", encoding="cp949")

# 2. 제거할 영업상태명 목록 정의
drop_statuses = ['폐업', '취소/말소/만료/정지/중지']

# 3. 비영업 상태 제거 → 영업 중인 업소만 남기기
df_filtered = df[~df['영업상태명'].isin(drop_statuses)].copy()

# 4. 자치구 추출 함수 정의 (도로명주소 또는 지번주소에서 '강남구' 등 뽑기)
def extract_gu(address):
    try:
        if pd.notnull(address) and "구" in address:
            return address.split()[1]
        else:
            return None
    except:
        return None

# 5. 도로명주소에서 자치구 추출
df_filtered['자치구'] = df_filtered['도로명주소'].apply(extract_gu)

# 6. 도로명주소에서 자치구 추출 실패한 경우 → 지번주소에서 추출
df_filtered['자치구'] = df_filtered['자치구'].fillna(df_filtered['지번주소'].apply(extract_gu))

# 7. 자치구별 업소 수 합산
gu_counts = df_filtered['자치구'].value_counts().reset_index()
gu_counts.columns = ['자치구', '청소년게임제공업소수']

# 8. 결과 확인 (선택)
print(gu_counts)

# 9. 결과 CSV 파일로 저장 (선택)
gu_counts.to_csv("서울시_자치구별_청소년게임시설.csv", index=False, encoding="utf-8-sig")

     자치구  청소년게임제공업소수
0    강서구          79
1    강북구          55
2    중랑구          46
3    동작구          46
4    송파구          43
5    노원구          43
6    강동구          41
7    관악구          36
8    구로구          35
9    금천구          29
10   은평구          27
11  영등포구          27
12   마포구          27
13   양천구          27
14   성북구          25
15   광진구          24
16   도봉구          19
17   강남구          17
18  동대문구          16
19   종로구          12
20   서초구          11
21   용산구           9
22  서대문구           9
23   성동구           6
24    중구           6


In [19]:
#서울시 일반게임제공업 인허가 정보.csv" 파일에서 폐업, 취소/말소/만료/정지/중지 상태를 제외하고,자치구 25개로 나눠서 업소 수를 합산
import pandas as pd

# 1. 데이터 불러오기 (한글 인코딩 고려)
df = pd.read_csv("서울시 일반게임제공업 인허가 정보.csv", encoding="cp949")

# 2. 폐업 및 취소/말소 등 영업 중이 아닌 상태 제거
drop_statuses = ['폐업', '취소/말소/만료/정지/중지']
df_filtered = df[~df['영업상태명'].isin(drop_statuses)].copy()

# 3. 자치구 추출 함수 정의 (도로명주소 또는 지번주소에서 "강남구" 등 추출)
def extract_gu(address):
    try:
        if pd.notnull(address) and "구" in address:
            return address.split()[1]  # 예: "서울특별시 강남구 테헤란로" → "강남구"
        else:
            return None
    except:
        return None

# 4. 도로명주소에서 자치구 추출
df_filtered['자치구'] = df_filtered['도로명주소'].apply(extract_gu)

# 5. 도로명주소에 자치구 없는 경우 → 지번주소에서 추출
df_filtered['자치구'] = df_filtered['자치구'].fillna(df_filtered['지번주소'].apply(extract_gu))

# 6. 자치구별 일반게임업소 수 합산
gu_counts = df_filtered['자치구'].value_counts().reset_index()
gu_counts.columns = ['자치구', '일반게임제공업소수']

# 7. 결과 확인
print(gu_counts)

# 8. 결과 저장 (선택)
gu_counts.to_csv("서울시_자치구별_일반게임시설.csv", index=False, encoding='utf-8-sig')

     자치구  일반게임제공업소수
0    관악구         61
1    구로구         45
2    강북구         27
3    종로구         19
4    도봉구         18
5    은평구         17
6   동대문구         16
7    성동구         15
8    강서구         15
9    금천구         14
10  영등포구         14
11   성북구         11
12   중랑구         10
13   강동구         10
14   송파구          9
15   서초구          7
16   노원구          7
17   광진구          6
18  서대문구          6
19    중구          5
20   동작구          5
21   양천구          4
22   강남구          2
23   마포구          1


In [21]:
#서울시 당구장업 인허가 정보.csv 파일에서 "영업상태명"이 "폐업"이나 "취소/말소/만료/정지/중지"인 데이터는 제거하고, 서울 자치구 25개로 나눠서 업소 수를 합산

import pandas as pd

# 1. CSV 파일 불러오기
df = pd.read_csv("서울시 당구장업 인허가 정보.csv", encoding="cp949")

# 2. 제외할 영업상태명 목록 정의
drop_statuses = ['폐업', '취소/말소/만료/정지/중지']

# 3. '폐업', '취소/말소/만료/정지/중지' 상태인 업소 제거 → 영업중인 업소만 남기기
df_filtered = df[~df['영업상태명'].isin(drop_statuses)].copy()

# 4. 자치구 추출 함수 정의
# 주소 문자열에서 "강남구", "마포구" 등 자치구 이름만 추출
def extract_gu(address):
    try:
        if pd.notnull(address) and "구" in address:
            return address.split()[1]  # 예: '서울특별시 강남구 테헤란로 1길' → '강남구'
        else:
            return None
    except:
        return None

# 5. 도로명주소에서 자치구 추출
df_filtered['자치구'] = df_filtered['도로명주소'].apply(extract_gu)

# 6. 도로명주소에서 자치구가 추출되지 않은 경우 → 지번주소에서 자치구 추출
df_filtered['자치구'] = df_filtered['자치구'].fillna(df_filtered['지번주소'].apply(extract_gu))

# 7. 자치구별 업소 수 합산
gu_counts = df_filtered['자치구'].value_counts().reset_index()
gu_counts.columns = ['자치구', '당구장업소수']

# 8. 결과 확인 (선택)
print(gu_counts)

# 9. 결과 저장 (선택)
gu_counts.to_csv("서울시_자치구별_당구장시설.csv", index=False, encoding="utf-8-sig")

     자치구  당구장업소수
0    서초구     196
1     중구     177
2    강남구     156
3    광진구     155
4    송파구     130
5    도봉구     120
6    강서구     118
7    성북구     117
8   영등포구     110
9    노원구     107
10   중랑구     107
11   강동구     102
12   강북구      98
13   은평구      97
14   금천구      92
15   마포구      89
16   양천구      87
17   구로구      86
18  동대문구      85
19   종로구      82
20   관악구      81
21   동작구      72
22   성동구      71
23  서대문구      61
24   용산구      34
